In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame,Series

import matplotlib.pyplot as plt

names = ["userid","time","bank_id","pre_amount_of_bill","pre_repayment","credit_amount",\
         "amount_of_bill_left","least_repayment","consume_amount","amount_of_bill","adjust_amount",\
         "circ_interest","avail_amount","prepare_amount","repayment_state"]

bill_train = pd.read_csv("../../pcredit/train/bill_detail_train.txt",header=None)
bill_test = pd.read_csv("../../pcredit/test/bill_detail_test.txt",header=None)

bill_data = pd.concat([bill_train,bill_test])
bill_data.columns=names
bill_data.head()

,userid,time,bank_id,pre_amount_of_bill,pre_repayment,credit_amount,amount_of_bill_left,least_repayment,consume_amount,amount_of_bill,adjust_amount,circ_interest,avail_amount,prepare_amount,repayment_state
0,3150,5906744363,6,18.626118,18.661937,20.664418,18.905766,17.847133,1,0.0,0.0,0.0,0.0,19.971271,0
1,3150,5906744401,6,18.905766,18.909954,20.664418,19.113305,17.911506,1,0.0,0.0,0.0,0.0,19.971271,0
2,3150,5906744427,6,19.113305,19.150290,20.664418,19.300194,17.977610,1,0.0,0.0,0.0,0.0,19.971271,0
3,3150,5906744515,6,19.300194,19.300280,21.000890,20.303240,18.477177,1,0.0,0.0,0.0,0.0,20.307743,0
4,3150,5906744562,6,20.303240,20.307744,21.000890,20.357134,18.510985,1,0.0,0.0,0.0,0.0,20.307743,0


In [2]:
#  1)获取消费笔数  
consume_amount = bill_data[['userid','consume_amount']]
consume_amount = pd.pivot_table(consume_amount,index=['userid'],values=['consume_amount'],aggfunc=np.sum)
consume_amount.head()

,consume_amount
userid,
2,58
3,4
4,2
5,0
6,122


In [5]:
consume_amount_cut  = consume_amount.copy()
consume_amount_cut['consume_amount'] = pd.cut(consume_amount_cut['consume_amount'],20,labels=range(1,21))
consume_amount_cut.head()

,consume_amount
userid,
2,1
3,1
4,1
5,1
6,1


In [6]:
# 信用卡额度 credit_amount
credit_amount = bill_data[['userid','credit_amount']]
credit_amount = pd.pivot_table(credit_amount,index=['userid'],values=['credit_amount'],aggfunc=np.mean)
credit_amount.head()

,credit_amount
userid,
2,18.638974
3,18.361833
4,16.055204
5,20.664418
6,20.567794


In [7]:
#  可用金额  avail_amount
avail_amount = bill_data[['userid','avail_amount']]
avail_amount = pd.pivot_table(avail_amount,index=['userid'],values=['avail_amount'],aggfunc=np.mean)
avail_amount.head()

,avail_amount
userid,
2,0.000000
3,0.000000
4,1.014342
5,0.000000
6,2.447743


In [8]:
# 预借现金额度  prepare_amount
prepare_amount = bill_data[['userid','prepare_amount']]
prepare_amount = pd.pivot_table(prepare_amount,index=['userid'],values=['prepare_amount'],aggfunc=np.mean)
prepare_amount.head()

,prepare_amount
userid,
2,7.087560
3,0.000000
4,7.808445
5,19.971271
6,13.788434


In [9]:
# 循环利息 circ_interest
circ_interest = bill_data[['userid','circ_interest']]
circ_interest = pd.pivot_table(circ_interest,index=['userid'],values=['circ_interest'],aggfunc=np.mean)
circ_interest.head()

,circ_interest
userid,
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0


In [10]:
datas = circ_interest.join([prepare_amount,avail_amount,consume_amount_cut,credit_amount])
datas.head()

,circ_interest,prepare_amount,avail_amount,consume_amount,credit_amount
userid,,,,,
2,0.0,7.087560,0.000000,1,18.638974
3,0.0,0.000000,0.000000,1,18.361833
4,0.0,7.808445,1.014342,1,16.055204
5,0.0,19.971271,0.000000,1,20.664418
6,0.0,13.788434,2.447743,1,20.567794


In [11]:
datas.to_csv("../data/train/bill_detail_1.csv")